In [1]:
# Стандартные библиотеки
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn.datasets import load_boston

# Предобработка данных и построение Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PolynomialFeatures, LabelEncoder, MaxAbsScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# Метрики классификации
from sklearn.metrics import classification_report

# Метрики регрессии

from sklearn.metrics import r2_score

# Модели регерессии
from sklearn.linear_model import Lasso, LinearRegression, Ridge

# Модели классификации
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")


In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
regressions = [ LinearRegression,
                Lasso,
                Ridge
              ]

In [6]:
for regression in regressions:
    start = time.time()
    model = regression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed = time.time() - start

    print('Модель: ', regression())
    print('r2_score: ', r2_score(y_test, y_pred))
    print('Время выполнения: ', round(elapsed,3))
    print('----','\n')

Модель:  LinearRegression()
r2_score:  0.6687594935356321
Время выполнения:  0.011
---- 

Модель:  Lasso()
r2_score:  0.6671453631686304
Время выполнения:  0.003
---- 

Модель:  Ridge()
r2_score:  0.6662221670168518
Время выполнения:  0.004
---- 



3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [7]:
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]

In [8]:
regressions = [ Lasso,
                Ridge
              ]

In [9]:
for regression in regressions:
    params = [{'alpha': alphas}]
    start = time.time()
    search = GridSearchCV(regression(), 
                          param_grid=params,
                          scoring='r2',
                          n_jobs=-1,
                          cv=5).fit(X_train, y_train)
    alpha = search.best_params_['alpha']
    model = regression(alpha).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed = time.time() - start

    print('Модель: ', regression())
    print('r2_score: ', r2_score(y_test, y_pred))
    print('alpha: ', alpha)
    print('Время выполнения: ', round(elapsed,3))
    print('----','\n')

Модель:  Lasso()
r2_score:  0.6687598638315152
alpha:  1e-05
Время выполнения:  3.843
---- 

Модель:  Ridge()
r2_score:  0.6687594856409728
alpha:  1e-05
Время выполнения:  0.044
---- 



Результат не изменился.

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
preprocessor = Pipeline([
                            ('std', StandardScaler()),
                            ('minmax', MinMaxScaler())
])

In [11]:
data_scale = preprocessor.fit_transform(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data_scale, y,
                                                    test_size=0.2,
                                                    random_state=RANDOM_STATE)

In [13]:
for regression in regressions:
    start = time.time()
    model = regression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed = time.time() - start

    print('Модель: ', regression())
    print('r2_score: ', r2_score(y_test, y_pred))
    print('Время выполнения: ', round(elapsed,3))
    print('----','\n')

Модель:  Lasso()
r2_score:  0.25739212837462977
Время выполнения:  0.002
---- 

Модель:  Ridge()
r2_score:  0.6780746612019258
Время выполнения:  0.001
---- 



Модель Ridge улучшила результат и по метрике и по скорости. Модель Lasso значительно ухудшила результат.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [14]:
for regression in regressions:
    params = [{'alpha': alphas}]
    start = time.time()
    search = GridSearchCV(regression(), 
                          param_grid=params,
                          scoring='r2',
                          n_jobs=-1,
                          cv=5).fit(X_train, y_train)
    alpha = search.best_params_['alpha']
    model = regression(alpha).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed = time.time() - start

    print('Модель: ', regression())
    print('r2_score: ', r2_score(y_test, y_pred))
    print('alpha: ', alpha)
    print('Время выполнения: ', round(elapsed,3))
    print('----','\n')

Модель:  Lasso()
r2_score:  0.668760771409485
alpha:  1e-05
Время выполнения:  0.049
---- 

Модель:  Ridge()
r2_score:  0.6702995459264809
alpha:  0.1
Время выполнения:  0.034
---- 



После подбора коэффициентов результаты Ridge не изменились, модель Lasso улучшила результат.

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [15]:
data_polynom = PolynomialFeatures(2).fit_transform(data_scale)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data_polynom, y,
                                                    test_size=0.2,
                                                    random_state=RANDOM_STATE)

In [17]:
for regression in regressions:
    start = time.time()
    model = regression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed = time.time() - start

    print('Модель: ', regression())
    print('r2_score: ', r2_score(y_test, y_pred))
    print('Время выполнения: ', round(elapsed,3))
    print('----','\n')

Модель:  Lasso()
r2_score:  0.2544868974438842
Время выполнения:  0.003
---- 

Модель:  Ridge()
r2_score:  0.8276750146248382
Время выполнения:  0.005
---- 



Модель Ridge значительно улучшила результат по метрике, Lasso ухудшила результат.

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [18]:
pipe = Pipeline([
                    ('scaler', StandardScaler()),
                    ('polynom', PolynomialFeatures()),
                    ('regressor', Ridge())
                ])

In [19]:
params_grid = {
                'scaler': [StandardScaler(), MinMaxScaler()],
                'polynom__degree': [2,3,4],
                'regressor': [Ridge(), Lasso()],
                'regressor__alpha': alphas
}

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=RANDOM_STATE)

In [ ]:
search = GridSearchCV(pipe, 
                      param_grid=params_grid,
                      scoring='r2',
                      n_jobs=-1,
                      cv=5).fit(X_train, y_train)


In [22]:
y_pred = search.predict(X_test)
print('Лучшие параметры: ', search.best_params_)
print('r2_score: ', r2_score(y_test, y_pred))
print('----','\n')

Лучшие параметры:  {'polynom__degree': 3, 'regressor': Lasso(alpha=0.001), 'regressor__alpha': 0.001, 'scaler': MinMaxScaler()}
r2_score:  0.8449734005131504
---- 



После подбора параметров модель Lasso значительно улучшила результат.

http://archive.ics.uci.edu/ml/datasets/Adult

In [23]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [24]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [25]:
# Создадим словарь для кодирования целевой переменной

target_map = {
                '<=50K' : 0,
                '>50K'  : 1
              }

In [26]:
# Заменим целевую переменную на числовые значения

data[14] = data[14].map(target_map)

In [27]:
# Разделим датасет на признаки и целевую переменную

X = data.iloc[:,0:14]
y = data[14]

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [28]:
X.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

Пропущенных значений нет.

10. Выберите колонки с числовыми и категориальными переменными.

In [29]:
# Определим категориальные и числовые признаки 

numeric_features = X.select_dtypes(include='int64').columns
categorical_features = X.select_dtypes(exclude='int64').columns

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [30]:
numeric_transformer = Pipeline([
                                ('Standart', StandardScaler()),
                                ('MinMax', MinMaxScaler())
                                ])

categorical_transformer = Pipeline([
                                    ('OneHot', OneHotEncoder())
                                    ])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
                  ])


In [31]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('Standart', StandardScaler()),
                                                 ('MinMax', MinMaxScaler())]),
                                 Int64Index([0, 2, 4, 10, 11, 12], dtype='int64')),
                                ('cat',
                                 Pipeline(steps=[('OneHot', OneHotEncoder())]),
                                 Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64'))])

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [32]:
X_processed = preprocessor.fit_transform(X)

In [33]:
X_processed.shape

(48842, 108)

In [34]:
classifiers = [
                LogisticRegression(max_iter=1000),
                LinearSVC(),
                SVC()
]       

In [35]:
for classifier in classifiers:
    start = time.time()
    classifier.fit(X_processed, y)
    score_accuracy = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='accuracy', cv=5)
    score_f1 = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='f1', cv=5)
    elapsed = time.time() - start

    print('Модель: ',classifier)
    print('Accuracy: ',score_accuracy.max())
    print('f1: ', score_f1.max())
    print('Время выполнения: ', round(elapsed,3))
    print('----', '\n')

Модель:  LogisticRegression(max_iter=1000)
Accuracy:  0.8564701064701065
f1:  0.6653937947494034
Время выполнения:  3.848
---- 

Модель:  LinearSVC()
Accuracy:  0.8572891072891073
f1:  0.6645813282001924
Время выполнения:  2.857
---- 

Модель:  SVC()
Accuracy:  0.845004095004095
f1:  0.6318840579710144
Время выполнения:  145.549
---- 



14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [36]:
X.isin(['?']).sum().sort_values(ascending=False)


6     2809
1     2799
13     857
0        0
2        0
3        0
4        0
5        0
7        0
8        0
9        0
10       0
11       0
12       0
dtype: int64

In [37]:
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')

In [38]:
X_imp = pd.DataFrame(imputer.fit_transform(X))

In [39]:
X_imp.isin(['?']).sum().sort_values(ascending=False)


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [40]:
X_processed = preprocessor.fit_transform(X_imp)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [41]:
for classifier in classifiers:
    start = time.time()
    classifier.fit(X_processed, y)
    score_accuracy = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='accuracy', cv=5)
    score_f1 = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='f1', cv=5)
    elapsed = time.time() - start

    print('Модель: ',classifier)
    print('Accuracy: ',score_accuracy.max())
    print('f1: ', score_f1.max())
    print('Время выполнения: ', round(elapsed,3))
    print('----', '\n')

Модель:  LogisticRegression(max_iter=1000)
Accuracy:  0.8561629811629812
f1:  0.6660328024720703
Время выполнения:  3.883
---- 

Модель:  LinearSVC()
Accuracy:  0.8542178542178542
f1:  0.6570327552986512
Время выполнения:  3.059
---- 

Модель:  SVC()
Accuracy:  0.8428542178542179
f1:  0.6266407389402042
Время выполнения:  143.023
---- 



Результаты моделей незначительно ухудшились.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [42]:
list_drop = X[(X=='?').sum(axis=1) > 0].index

In [43]:
data_drop = data.drop(index = list_drop)

In [44]:
X = data_drop.iloc[:,0:14]
y = data_drop[14]

In [45]:
X_processed = preprocessor.fit_transform(X)

In [46]:
for classifier in classifiers:
    start = time.time()
    classifier.fit(X_processed, y)
    score_accuracy = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='accuracy', cv=5)
    score_f1 = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='f1', cv=5)
    elapsed = time.time() - start

    print('Модель: ',classifier)
    print('Accuracy: ',score_accuracy.max())
    print('f1: ', score_f1.max())
    print('Время выполнения: ', round(elapsed,3))
    print('----', '\n')

Модель:  LogisticRegression(max_iter=1000)
Accuracy:  0.854046881910659
f1:  0.6721311475409836
Время выполнения:  3.785
---- 

Модель:  LinearSVC()
Accuracy:  0.8539363113666519
f1:  0.6701622971285892
Время выполнения:  3.263
---- 

Модель:  SVC()
Accuracy:  0.8427686864219371
f1:  0.6389029964448958
Время выполнения:  133.054
---- 



Результаты практически не изменились.

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [47]:
classifiers = [
                RandomForestClassifier(),
                GradientBoostingClassifier()
                ]

In [48]:
for classifier in classifiers:
    start = time.time()
    classifier.fit(X_processed, y)
    score_accuracy = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='accuracy', cv=5)
    score_f1 = cross_val_score(classifier, X_processed, y, n_jobs=-1, scoring='f1', cv=5)
    elapsed = time.time() - start

    print('Модель: ',classifier)
    print('Accuracy: ',score_accuracy.max())
    print('f1: ', score_f1.max())
    print('Время выполнения: ', round(elapsed,3))
    print('----', '\n')

Модель:  RandomForestClassifier()
Accuracy:  0.8530517470145953
f1:  0.6828681424446583
Время выполнения:  70.567
---- 

Модель:  GradientBoostingClassifier()
Accuracy:  0.8674259177355153
f1:  0.6943877551020409
Время выполнения:  14.656
---- 



GradientBoostingClassifier показал лучший результат.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [49]:
numeric_transformer = Pipeline(steps=[
                                ('scaler', StandardScaler()),
                                ])

categorical_transformer = Pipeline([
                                    ('encoder', OneHotEncoder())
                                    ])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
                  ])

In [50]:
pipe = Pipeline([
                    ('preprocessor', preprocessor),
                    ('classifier', GradientBoostingClassifier())
                ])

In [51]:
params_grid = {
                'preprocessor__num__scaler': [StandardScaler(with_mean=False), MaxAbsScaler()],
                'preprocessor__cat__encoder': [OneHotEncoder(handle_unknown='ignore'), LabelEncoder()],
                'classifier': [RandomForestClassifier(), GradientBoostingClassifier(), LogisticRegression(max_iter=1000), LinearSVC(), SVC()]

                }



In [52]:
search = GridSearchCV(pipe, 
                      param_grid=params_grid,
                      scoring='accuracy',
                      n_jobs=-1,
                      cv=5).fit(X, y)

/Users/matdiv/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/matdiv/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/matdiv/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/matdiv/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/matdiv/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [53]:
print('Лучшие параметры: ', search.best_params_)

Лучшие параметры:  {'classifier': GradientBoostingClassifier(), 'preprocessor__cat__encoder': OneHotEncoder(handle_unknown='ignore'), 'preprocessor__num__scaler': MaxAbsScaler()}


In [54]:
data_drop = data.drop(index = list_drop)
X = data_drop.iloc[:,0:14]
y = data_drop[14]

In [55]:
numeric_transformer = Pipeline(steps=[
                                ('scaler', MaxAbsScaler()),
                                ])

categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore'))])


preprocessor_optim = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
                  ])

In [56]:
X_preproc_optim = preprocessor_optim.fit_transform(X)

In [57]:
pipe_opt = Pipeline(steps=[
                        ('preprocessor_optim', preprocessor_optim),
                        ('classifier', GradientBoostingClassifier())
])

In [58]:
pipe_opt

Pipeline(steps=[('preprocessor_optim',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MaxAbsScaler())]),
                                                  Int64Index([0, 2, 4, 10, 11, 12], dtype='int64')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Int64Index([1, 3, 5, 6, 7, 8, 9, 13], dtype='int64'))])),
                ('classifier', GradientBoostingClassifier())])

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=RANDOM_STATE)

In [60]:
model = pipe_opt.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      6745
           1       0.81      0.61      0.70      2300

    accuracy                           0.87      9045
   macro avg       0.84      0.78      0.81      9045
weighted avg       0.86      0.87      0.86      9045

